In [ ]:
import uos
import machine, onewire, ds18x20
import utime
from machine import Pin, UART
import time
wifi = UART(1, baudrate=9600, tx=Pin(4), rx=Pin(5))
rs = machine.Pin(10,machine.Pin.OUT)
e = machine.Pin(11,machine.Pin.OUT)
d4 = machine.Pin(12,machine.Pin.OUT)
d5 = machine.Pin(13,machine.Pin.OUT)
d6 = machine.Pin(14,machine.Pin.OUT)
d7 = machine.Pin(15,machine.Pin.OUT)

pt1 = machine.Pin(8,machine.Pin.OUT)
pt2 = machine.Pin(9,machine.Pin.OUT)

buz = machine.Pin(7,machine.Pin.OUT)
ir = machine.Pin(6,machine.Pin.IN)

pt1.value(0)
pt2.value(0)

def pulseE():
    e.value(1)
    utime.sleep_us(40)
    e.value(0)
    utime.sleep_us(40)
def send2LCD4(BinNum):
    d4.value((BinNum & 0b00000001) >>0)
    d5.value((BinNum & 0b00000010) >>1)
    d6.value((BinNum & 0b00000100) >>2)
    d7.value((BinNum & 0b00001000) >>3)
    pulseE()
def send2LCD8(BinNum):
    d4.value((BinNum & 0b00010000) >>4)
    d5.value((BinNum & 0b00100000) >>5)
    d6.value((BinNum & 0b01000000) >>6)
    d7.value((BinNum & 0b10000000) >>7)
    pulseE()
    d4.value((BinNum & 0b00000001) >>0)
    d5.value((BinNum & 0b00000010) >>1)
    d6.value((BinNum & 0b00000100) >>2)
    d7.value((BinNum & 0b00001000) >>3)
    pulseE()
def setUpLCD():
    rs.value(0)
    send2LCD4(0b0011)#8 bit
    send2LCD4(0b0011)#8 bit
    send2LCD4(0b0011)#8 bit
    send2LCD4(0b0010)#4 bit
    send2LCD8(0b00101000)#4 bit,2 lines?,5*8 bots
    send2LCD8(0b00001100)#lcd on, blink off, cursor off.
    send2LCD8(0b00000110)#increment cursor, no display shift
    send2LCD8(0b00000001)#clear screen
    utime.sleep_ms(2)#clear screen needs a long delay

setUpLCD()
rs.value(1)
for x in '  WELCOME':
    send2LCD8(ord(x))



ds_pin = machine.Pin(26)
ds_sensor = ds18x20.DS18X20(onewire.OneWire(ds_pin))
roms = ds_sensor.scan()
cnt=0
while True:
    print('Found DS devices')
    print('Temperature')
    roms = ds_sensor.scan()
    ds_sensor.convert_temp()
    utime.sleep(1)
    for rom in roms:
      temperature_Celsius = round(ds_sensor.read_temp(rom), 2)
      print(temperature_Celsius)

    utime.sleep(3)
    setUpLCD()
    rs.value(1)
    if(ir.value()==1):
        for x in 'T:'+str(temperature_Celsius) + '   OPEN':
            send2LCD8(ord(x))
        buz.value(1)
        time.sleep(0.05)
        buz.value(0)

    if(ir.value()==0):
        for x in 'T:'+str(temperature_Celsius) + '   CLOSED':
            send2LCD8(ord(x))




    if(temperature_Celsius<25):
        pt1.value(0)
        pt2.value(0)

        rs.value(0)
        time.sleep(0.02)
        send2LCD8(0b11000000)#
        time.sleep(0.02)
        rs.value(1)
        for x in 'COOL-OFF':
            send2LCD8(ord(x))
    else if(temperature_Celsius>30):
        pt1.value(1)
        pt2.value(0)


        rs.value(0)
        time.sleep(0.02)
        send2LCD8(0b11000000)#
        time.sleep(0.02)
        rs.value(1)
        for x in 'COOL-ON':
            send2LCD8(ord(x))

    cnt=cnt+1
    if(cnt>5):
        cnt=0
        print('updating')
        wifi.write("685012,E8VRLJ4COTBHSMXQ,0,0,SRC 24G,src@internet,"+str(temperature_Celsius)+",0\n")